In [1]:
from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB
from astropy.table import QTable
import astropy.units as u

#import pandas as pd
import numpy as np
import requests
from pprint import pprint
from array import *

# Define access to JPL SBDB

In [2]:
def get_dict(name_list):
    # Return information for each bodies in name_list from JPL Small Body Database in form of dictionair
    # INPUT
    # name_list      list [name1, name2, ..., nameN]
    # OUTPUT
    # dict_bodies    dict with the following structure
    # str(name): "fullname"
    #         "our_id"                is the id inside the new dict, from 0 to N-1 where N is the name_list length
    #         "neo_flag"
    #         "orbit_class"
    #         "pha_flag"
    #         "object_kind"
    #         "moid"
    #         "condition_code"
    #         "rms"
    #         "orbit_comment"
    #         "magn_radius_flag"      is "H" if the next parameter is the magnitude, is "D" if the next parameter is the diameter
    #         "H" or "D"
    #         "spectral_category_flag"     T, S or 0
    #         "spectral_category"
    #         "impacts": str(impact id):   'width'  
    #                                      'energy'
    #                                      'stretch'
    #                                      'ip'
    #                                      'dt'
    #                                      'date'
    #                                      'sigma_lov'
    #                                      'h'
    #                                      'mass'
    #                                      'v_inf'
    #                                      'sigma_imp'
    #                                      'method'
    #                                      'ts'
    #                                      'diam'
    #                                      'dist'
    #                                      'v_imp'
    #                                      'ps'
    
    our_id=0;
    dict_bodies={};

    for name in name_list:
        sbdb = SBDB.query(name, neo_only=True, full_precision=True, phys=True, virtual_impactor=True)
        asteroid={"fullname": sbdb["object"]["fullname"],# TODO vogliamo fare un check?
          "our_id":our_id,
          "neo_flag": sbdb["object"]["neo"],
          "orbit_class":sbdb["object"]["orbit_class"]["code"],
          "pha_flag":sbdb["object"]["pha"],
          "object_kind":sbdb["object"]["kind"], #an asteroid numbered au unbered asteroid (cn, cu for comet)
          "moid": sbdb["orbit"]["moid"],
          "condition_code": sbdb["orbit"]["condition_code"], #OCC
          "rms": sbdb["orbit"]["rms"],
          "orbit_comment":sbdb["orbit"]["comment"],
         };
        try:     
            asteroid["magn_radius_flag"]='H';
            asteroid["H"]=sbdb['phys_par']['H'];
        except:
            asteroid["magn_radius_flag"]='D';
            asteroid["D"]=sbdb['phys_par']['diameter'];
        asteroid["impacts"]={}
        flag_bool=1;
        if 'phys_par' in sbdb.keys():
            spect_flag=0;
            if 'spec_T' in sbdb['phys_par'].keys():
                asteroid["spectral_category_flag"]='T'
                asteroid["spectral_category"]=sbdb['phys_par']['spec_T']
                spect_flag=1;
            if 'spec_B' in sbdb['phys_par'].keys():
                asteroid["spectral_category_flag"]='B'
                asteroid["spectral_category"]=sbdb['phys_par']['spec_B']
                spect_flag=1;
            if spect_flag==0:
                asteroid["spectral_category_flag"]='0'
        else:
            asteroid["spectral_category"]='0'
                
        if 'ip' in sbdb["vi_data"]:
            n_imp=len(sbdb["vi_data"]['ip']);
            for key in sbdb["vi_data"].keys():
                if flag_bool==1:
                    for i in range(0,n_imp):
                        asteroid["impacts"][str(i)]={};
                    flag_bool=0;
                for i in range(0,n_imp): 
                    try:
                        asteroid["impacts"][str(i)][key]=sbdb["vi_data"][key][i];
                    except:
                        pprint(name+" could raise error in importing virtual impact data") #this exception is raised if only one impact is present
        dict_bodies[name]=asteroid;
        our_id=our_id+1;
        flag_bool=1;
        del asteroid;
    return dict_bodies
        

# Extract ESA risk list

In [3]:
def extract_esa_name_from_file(file_name):
    f = open(file_name, "r")
    line=f.readline()
    line=f.readline()
    line=f.readline()
    line=f.readline()
    counter=0;
    esa_names=[]
    for line in f:
        word="";
        for c in line:
            if c==' ':
                break;
            else:
                word=word+c;
        esa_names.append(word);  
    return esa_names

In [4]:
esa_risk_names=extract_esa_name_from_file("esa_risk_list.txt");

# Extract SENTRY risk list

In [5]:
def get_sentry_risk_list():
    url = 'https://ssd-api.jpl.nasa.gov/sentry.api'
    r = requests.get(url)
    data = r.json()
    sentry_risk_names=[];
    for i in range(0,len(data['data'])):
        name_=""
        name=data['data'][i]['des'];
        for c in name:
            if c==' ':
                a=0;
            else:
                name_=name_+c;
        sentry_risk_names.append(name_)
    return sentry_risk_names


In [6]:
sentry_risk_names=get_sentry_risk_list()

# Merge risk lists

In [7]:
risk_list=esa_risk_names;
counter=0;
for risk_name in sentry_risk_names:
    if risk_name not in esa_risk_names:
        risk_list.append(risk_name)

# Query SBDB for all risk list elements

In [8]:
dict_risk_list=get_dict(risk_list);

'29075 could raise error in importing virtual impact data'


# Select only asteroids with info on spectral type

In [9]:
spectral_known=[];
for key in dict_risk_list.keys():
    if (dict_risk_list[key]['spectral_category_flag']=="B") or (dict_risk_list[key]['spectral_category_flag']=="T"):
        spectral_known.append('key');
        pprint(key)

'101955'


# Requirements filter

# MOID<=0.05au, H<=22 (if H is not available diameter>=400)

In [10]:
MOID_H_selected=[];
for key in dict_risk_list.keys():
    if float(dict_risk_list[key]['moid'].scale)<=0.05: #MOID<=0.05 AU
        if (dict_risk_list[key]["magn_radius_flag"]=='H' and float(dict_risk_list[key]["H"])<=22) or (dict_risk_list[key]["magn_radius_flag"]=='D' and float(dict_risk_list[key]["D"])>=400):
            MOID_H_selected.append(key)

# At least one impact 2026<year<22048 with a Palermo Scale>=-5

In [11]:
date_selected=[];
PS_date_selected=[]
for key in dict_risk_list.keys():
    if '0' in dict_risk_list[key]['impacts'].keys():
        max_P=-100;
        date_flag=0;
        for imp_id in dict_risk_list[key]['impacts'].keys():
            word='';
            for c in dict_risk_list[key]['impacts'][imp_id]['date']:
                #pprint(c)
                if c=='-':
                    break;
                else:
                    word=word+c;
            if int(word)<2048 and int(word)>2026:
                date_flag=1;
                if float(dict_risk_list[key]['impacts'][imp_id]['ps'])>max_P:
                    max_P=float(dict_risk_list[key]['impacts'][imp_id]['ps']);
        if date_flag==1:
            date_selected.append(key)
            if max_P>=-5:
                PS_date_selected.append(key)

# Intersect filtered lists

In [12]:
list(set(PS_date_selected) & set(MOID_H_selected))

['2007FT3', '2016NL56']